# Ex2 - Getting and Knowing your Data

Check out [Chipotle Exercises Video Tutorial](https://www.youtube.com/watch?v=lpuYZ5EUyS8&list=PLgJhDSE2ZLxaY_DigHeiIDC1cD09rXgJv&index=2) to watch a data scientist go through the exercises

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [34]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import pyspark.sql.functions as F

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).

In [20]:
import urllib
path = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
urllib.request.urlretrieve(path, "./data.csv")

('./data.csv', <http.client.HTTPMessage at 0x7b1b1499a290>)

### Step 3. Assign it to a variable called chipo.

In [94]:
chipo = spark.read.csv('./data.csv', sep=r'\t', header=True, inferSchema=True)
chipo.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

### Step 4. See the first 10 entries

In [54]:
chipo.limit(10).show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [55]:
# Solution 1
chipo.count()

4622

### Step 6. What is the number of columns in the dataset?

In [56]:
len(chipo.columns)

5

### Step 7. Print the name of all the columns.

In [57]:
chipo.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. How is the dataset indexed?

### Step 9. Which was the most-ordered item?

In [58]:
chipo.groupBy('item_name').agg(F.sum('quantity'))\
     .withColumnRenamed('sum(quantity)', 'quantity')\
     .orderBy('quantity', ascending=False)\
     .limit(1).select('item_name').show()

+------------+
|   item_name|
+------------+
|Chicken Bowl|
+------------+



### Step 10. For the most-ordered item, how many items were ordered?

In [59]:
chipo.groupBy('item_name').agg(F.sum('quantity'))\
     .withColumnRenamed('sum(quantity)', 'quantity')\
     .orderBy('quantity', ascending=False)\
     .limit(1).show()

+------------+--------+
|   item_name|quantity|
+------------+--------+
|Chicken Bowl|     761|
+------------+--------+



### Step 11. What was the most ordered item in the choice_description column?

In [61]:
chipo.filter(F.col('choice_description') != 'NULL')\
     .groupBy('choice_description')\
     .agg(F.sum('quantity'))\
     .withColumnRenamed('sum(quantity)', 'quantity')\
     .orderBy('quantity', ascending=False)\
     .limit(1).show(truncate=False)

+------------------+--------+
|choice_description|quantity|
+------------------+--------+
|[Diet Coke]       |159     |
+------------------+--------+



### Step 12. How many items were orderd in total?




In [62]:
chipo.agg(F.sum('quantity')).show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [95]:
chipo.schema

StructType([StructField('order_id', IntegerType(), True), StructField('quantity', IntegerType(), True), StructField('item_name', StringType(), True), StructField('choice_description', StringType(), True), StructField('item_price', StringType(), True)])

#### Step 13.b. Create a lambda function and change the type of item price

In [96]:
from pyspark.sql.types import FloatType
chipo = chipo.withColumn('item_price', F.split(chipo['item_price'], '\$')[1].cast(FloatType()))
chipo.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|      2.39|
|       1|       1|                Izze|        [Clementine]|      3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39|
|       1|       1|Chips and Tomatil...|                NULL|      2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|
|       3|       1|       Side of Chips|                NULL|      1.69|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|     11.75|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|      9.25|
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|      9.25|
|       5|       1| Chips and Guacamole|           

#### Step 13.c. Check the item price type

In [101]:
chipo.schema['item_price'].dataType

FloatType()

### Step 14. How much was the revenue for the period in the dataset?

In [107]:
revenue = chipo.agg(F.sum('item_price')).select('sum(item_price)').collect()
revenue_value = revenue[0]['sum(item_price)']
print(f"The revenue was ${revenue_value}")

The revenue was $34500.159754514694


### Step 15. How many orders were made in the period?

In [108]:
chipo.select('order_id').distinct().count()

1834

### Step 16. What is the average revenue amount per order?

In [113]:
chipo.groupBy('order_id').agg(F.sum(F.col('item_price') * F.col('quantity')))\
     .groupBy('order_id').agg(F.avg('sum((item_price * quantity))'))\
     .agg(F.avg('avg(sum((item_price * quantity)))')).show()

+--------------------------------------+
|avg(avg(sum((item_price * quantity))))|
+--------------------------------------+
|                    21.394231042919138|
+--------------------------------------+



In [116]:
chipo.createOrReplaceTempView('chipo')
spark.sql("""
    with total_price as
      (select order_id, sum(item_price * quantity) total from chipo group by order_id),
    avg_by_order as
      (select order_id, avg(total) total_avg from total_price group by order_id)
    select avg(total_avg) from avg_by_order
""").show()

+------------------+
|    avg(total_avg)|
+------------------+
|21.394231042919138|
+------------------+



### Step 17. How many different items are sold?

In [117]:
chipo.select('item_name').distinct().count()

50